### Preparation

In [178]:
using BenchmarkTools

using NBInclude;
@nbinclude("Rosenbrock_BFGS.ipynb");
@nbinclude("Rosenbrock_GD_Newton.ipynb");

[4.35903187, 0.88508975, 0.71357297, 0.5682926, 0.39783956, 0.28681072, 0.2446889, 0.00662869, 0.00630391, 0.00145395]
[40.6137926, 4.62894776, 4.62403037, 3.90296864, 3.59653136, 1.41797659, 1.3939231, 1.1102141, 0.81889474, 0.64550908]
function value in iteration 1 for x=[-58.21493943098327, 44.765612379044214] is 1.1183799402817593e9
function value in iteration 2 for x=[7.78733361641749e7, 668887.4777271803] is 3.6775206711858544e33
function value in iteration 3 for x=[-1.8889755352658453e26, 1.2128512969346094e18] is 1.2732255147981023e107
function value in iteration 4 for x=[2.6961185913271544e81, 7.136457145665774e54] is Inf
function value in iteration 5 for x=[-7.839294225219906e246, 1.4538110916999838e165] is Inf
function value in iteration 5666 for x=[0.9968403531725719, 0.9936780220224787] is 9.999415112942938e-6
function value in iteration 1 for x=[0.09273646170588251, 0.008571907510431576] is 0.823127207125418
function value in iteration 2 for x=[0.9949218117077823, 0.17593

In [169]:
function rosenbrock_performance(fun, samples, dimensions, max_iters, max_error=0)
    timed_history = []
    for j in 1:samples
        timed = @timed fun(rand(dimensions), max_iters, max_error)
        push!(timed_history, timed)
    end
    
    mean_iters = sum(map(h->h.value[3], timed_history))/samples
    mean_error = sum(map(h->last(h.value[2]), timed_history))/samples
    mean_exec_time = sum(map(h->h.time, timed_history))/samples
    mean_alloc_bytes = sum(map(h->h.bytes, timed_history))/samples
    
    return mean_iters, mean_error, mean_exec_time, mean_alloc_bytes
end

rosenbrock_performance (generic function with 2 methods)

In [179]:
SAMPLES = 10;
DIMENS = 3;
ITERS = 10000;
ERROR = 10e-6;

rate = 0.001953125; #GradientDescent
m = 4; #L-BFGS

### Gradient Descent performance

In [180]:
gd(x, max_iters, max_error) = rosenbrock_gradient_descent(x, rate, max_iters, max_error)
rosenbrock_performance(gd, SAMPLES, DIMENS, ITERS, ERROR)

(10000.0, 0.6086631932049913, 0.0198483748, 1.11247479e7)

### Newtons' Method performance

In [181]:
rosenbrock_performance(rosenbrock_newton, SAMPLES, DIMENS, ITERS, ERROR)

(9.5, 4.102236698561272e-7, 0.0722284525, 1.43692278e7)

### BFGS performance

In [182]:
rosenbrock_performance(rosenbrock_BFGS, SAMPLES, DIMENS, ITERS, ERROR)

(15.3, 3.273e-6, 0.0062869816, 1.1553466e6)

### L-BFGS performance

In [183]:
lbfgs(x, max_iters, max_error) = rosenbrock_LBFGS(x, m, max_iters, max_error)
rosenbrock_performance(lbfgs, SAMPLES, DIMENS, ITERS, ERROR)

(18.7, 3.5139999999999997e-6, 0.0056819626, 1.1381331e6)